# Tank with Holes

In [1]:
import sys
sys.path.append('../')

In [2]:
from tank import Tank, Hole, create_holes
import ipywidgets as widgets
from ipycanvas import canvas
from IPython.display import display, Markdown, Latex
import time
from demo import *
from pythreejs import *

In [ ]:
%matplotlib widget

In [ ]:
%%capture
c = canvas.Canvas(width=500, height=500)
#c.layout.height = "75%"
m = Tank(create_holes(25, 2), 0.06, c=c)
cam = PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, matrixWorldNeedsUpdate=True, position=(2.5, 5.0, 5.0), quaternion=(0.0, 0.0, 0.0, 1.0), rotation=(0.0, 0.0, 0.0, 'XYZ'), scale=(1.0, 1.0, 1.0), shadow=DirectionalLightShadow(camera=OrthographicCamera(bottom=-5.0, far=500.0, left=-5.0, near=0.5, position=(0.0, 0.0, 0.0), projectionMatrix=(0.2, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, -0.004004004004004004, 0.0, 0.0, 0.0, -1.002002002002002, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), right=5.0, rotation=(0.0, 0.0, 0.0, 'XYZ'), scale=(1.0, 1.0, 1.0), top=5.0, up=(0.0, 1.0, 0.0)), mapSize=(512.0, 512.0)), target=Object3D(position=(0.0, 0.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), rotation=(0.0, 0.0, 0.0, 'XYZ'), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), up=(0.0, 1.0, 0.0)),), position=(-2.5825074114212176, -0.20262044568016213, -1.5268626747956473), projectionMatrix=(2.1445069205095586, 0.0, 0.0, 0.0, 0.0, 2.1445069205095586, 0.0, 0.0, 0.0, 0.0, -1.00010000500025, -1.0, 0.0, 0.0, -0.200010000500025, 0.0), quaternion=(0.0043975709080152445, 0.8744994255096968, 0.007930175133756838, -0.48494177843908964), rotation=(-2.852815661258447, -0.9323906225151685, -2.907399997450749, 'XYZ'), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0))
scene = Scene(children=[m.tank_pivot, m.water_pivot, cam, AmbientLight(color='#FFFFFF')], background=None)
m.set_threejs_scene(scene)

renderer = Renderer(camera=cam,
                    #controls=[OrbitControls(controlling=cam)],
                    scene=scene,
                    alpha=True,
                    clearOpacity=0.0,
                    clearColor='gray',
                    width=500, height=500)
m.draw_holes3D(scene)
demo = Demo(m, drawable=widgets.HBox([c, widgets.HTML("&nbsp;" * 5), renderer, m.plot_box]))

In [4]:
%%capture
c = canvas.Canvas(width=500, height=500)
#c.layout.height = "75%"
m = Tank(create_holes(25, 2), 0.06, c=c)
cam = PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, matrixWorldNeedsUpdate=True, position=(2.5, 5.0, 5.0), quaternion=(0.0, 0.0, 0.0, 1.0), rotation=(0.0, 0.0, 0.0, 'XYZ'), scale=(1.0, 1.0, 1.0), shadow=DirectionalLightShadow(camera=OrthographicCamera(bottom=-5.0, far=500.0, left=-5.0, near=0.5, position=(0.0, 0.0, 0.0), projectionMatrix=(0.2, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, -0.004004004004004004, 0.0, 0.0, 0.0, -1.002002002002002, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), right=5.0, rotation=(0.0, 0.0, 0.0, 'XYZ'), scale=(1.0, 1.0, 1.0), top=5.0, up=(0.0, 1.0, 0.0)), mapSize=(512.0, 512.0)), target=Object3D(position=(0.0, 0.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), rotation=(0.0, 0.0, 0.0, 'XYZ'), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), up=(0.0, 1.0, 0.0)),), position=(-2.5825074114212176, -0.20262044568016213, -1.5268626747956473), projectionMatrix=(2.1445069205095586, 0.0, 0.0, 0.0, 0.0, 2.1445069205095586, 0.0, 0.0, 0.0, 0.0, -1.00010000500025, -1.0, 0.0, 0.0, -0.200010000500025, 0.0), quaternion=(0.0043975709080152445, 0.8744994255096968, 0.007930175133756838, -0.48494177843908964), rotation=(-2.852815661258447, -0.9323906225151685, -2.907399997450749, 'XYZ'), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0))
scene = Scene(children=[m.tank_pivot, m.water_pivot, cam, AmbientLight(color='#FFFFFF')], background=None)
m.set_threejs_scene(scene)

renderer = Renderer(camera=cam,
                    #controls=[OrbitControls(controlling=cam)],
                    scene=scene,
                    alpha=True,
                    clearOpacity=0.0,
                    clearColor='gray',
                    width=500, height=500)
m.draw_holes3D(scene)
demo = Demo(m, drawable=widgets.HBox([c, widgets.HTML("&nbsp;" * 5), renderer, m.plot_box]))

In [5]:
faucetbox = BoxBufferGeometry(1, 0.5, 1)
faucetbox_mesh = Mesh(faucetbox, material=MeshPhongMaterial(color='gray'), position=[0, 0.25, 1])

faucetpipe = BoxBufferGeometry(0.5, 0.25, 0.75)
faucetpipe_mesh = Mesh(faucetpipe, material=MeshPhongMaterial(color='black'), position=[0, 0.25, 0])

stream = CylinderBufferGeometry(0.1, 0.1, 1.2, 16, 8)
stream_mesh = Mesh(stream, material=MeshPhongMaterial(color='lightblue'), position=[0, -0.5, -0.25])

In [6]:
scene.add(faucetbox_mesh)
scene.add(faucetpipe_mesh)
scene.add(stream_mesh)

In [7]:
demo.show()

Output()

HTML(value="<div class='seperator'></div> <br />")

HTML(value="<div class='seperator'></div>")

Output()

In [8]:
c.on_client_ready(m.draw)

In [9]:
tank_pivot = Object3D()
tank_pivot.scale = [1, 1, 1]
tank_pivot.position = [0, -1, 0]
tank_pivot.rotation = [10, 0, 0, tank_pivot.rotation[-1]]

In [10]:
tank_pivot = Object3D(rotation=[0, 0, 0, 'XYZ'])
tank_pivot.scale = [1, 1, 1]
tank_pivot.position = [0, -1, 0]

In [11]:
m = Mesh(BufferGeometry(), rotation=[0, 5, 0, 'XYZ'])

In [12]:
m.rotation = [10, 0, 0, 'XYZ']

In [13]:
tank_pivot = Object3D(rotation=[0, 0, 0, 'XYZ'])
tank_pivot.scale = [1, 1, 1]
tank_pivot.position = [0, -1, 0]

In [14]:
m = Mesh(BufferGeometry(), rotation=[0, 5, 0, 'XYZ'])

In [15]:
m.rotation = [10, 0, 0, 'XYZ']